If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [1]:
! pip install datasets transformers
!apt install git-lfs

You should consider upgrading via the 'c:\users\redux gamer\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
'apt' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
import transformers
from datasets import DatasetDict, ClassLabel, load_dataset, load_metric
import os
import random
import pandas as pd
from IPython.display import display, HTML

# from google.colab import drive
# drive.mount('/content/drive')

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

Then you need to install Git-LFS. Uncomment the following instructions:

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

# Fine-tuning a model on a multiple choice task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a multiple choice task, which is the task of selecting the most plausible inputs in a given selection. The dataset used here is [SWAG](https://www.aclweb.org/anthology/D18-1009/) but you can adapt the pre-processing to any other multiple choice dataset you like, or your own data. SWAG is a dataset about commonsense reasoning, where each example describes a situation then proposes four options that could go after it. 

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a mutiple choice head. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those two parameters, then the rest of the notebook should run smoothly:

In [3]:
model_checkpoint = "bert-base-uncased"#"bert-base-uncased"
batch_size = 8

## Loading the dataset

In [4]:
Mutual = pd.read_csv(os.path.join(os.getcwd(),'MuTual_dataset.csv'))
# swag = pd.read_csv(os.path.join(os.getcwd(),'swag.csv'))
# swag = swag.rename(columns={'ending0': 'option1',
#  'ending1': 'option2',
#  'ending2': 'option3',
#  'ending3': 'option4',
#  'sent1':'article',
#  'video-id':'id'})

# swag['option1'] = swag['sent2'] +' '+ swag['option1']
# swag['option2'] = swag['sent2'] +' '+ swag['option2']
# swag['option3'] = swag['sent2'] +' '+ swag['option3']
# swag['option4'] = swag['sent2'] +' '+ swag['option4']
# swag = swag[swag['scope'] != 'test']

# swag = swag.drop(columns=['fold-ind', 'gold-source', 'startphrase', 'sent2'])

Mutual['answers'] = Mutual['answers'].replace(to_replace={'A':0, 'B':1, 'C':2, 'D':3})
Mutual['dataset'] = 'mutual'
Mutual = Mutual.rename(columns={'answers':'label'})
train_dataset = Mutual[Mutual['scope'] =='train']#.append(swag)
val_dataset = Mutual[Mutual['scope'] =='dev']#.head(10)

train_dataset.to_csv(os.path.join(os.getcwd(),'Train_Mutual_SWAG.csv'), index=False)

val_dataset.to_csv(os.path.join(os.getcwd(),'Validation_Mutual.csv'), index=False)
# train_dataset=train_dataset.head(10)
# val_dataset=val_dataset.head(10)
train_dataset = DatasetDict.from_csv(os.path.join(os.getcwd(),'Train_Mutual_SWAG.csv'))
val_dataset = DatasetDict.from_csv(os.path.join(os.getcwd(),'Validation_Mutual.csv'))


Using custom data configuration default-9f0a21075db12725


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Using custom data configuration default-4d13b14f5ebadb1f


Dataset csv downloaded and prepared to C:\Users\Redux Gamer\.cache\huggingface\datasets\csv\default-9f0a21075db12725\0.0.0. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to C:\Users\Redux Gamer\.cache\huggingface\datasets\csv\default-4d13b14f5ebadb1f\0.0.0. Subsequent calls will reuse this data.


We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data. This can be easily done with the functions `load_dataset`.  

In [5]:
train_dataset

Dataset({
    features: ['label', 'article', 'id', 'scope', 'option1', 'option2', 'option3', 'option4', 'dataset'],
    num_rows: 7088
})

The `dataset` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set (with more keys for the mismatched validation and test set in the special case of `mnli`).

To access an actual element, you need to select a split first, then give an index:

In [6]:
train_dataset[0]

{'label': 0,
 'article': "m : do you want to sit under that tree , granny ? it wo n't be so hot there . f : i 'd prefer to sit in the sun . it looks nice there by the river . m : shall i put the chairs there then ? f : yes , there in the car . why do n't you go and get them ?",
 'id': 'train_5792',
 'scope': 'train',
 'option1': "m : i will , granny . i agree with your idea . let 's sit here by the river .",
 'option2': 'm : okay . it is cold outside , we just stay in the car .',
 'option3': "m : i do n't want to , because it looks bad there by the river .",
 'option4': 'm : sure . it will be hot under that tree .',
 'dataset': 'mutual'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [7]:

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(train_dataset)

,label,article,id,scope,option1,option2,option3,option4,dataset
0,0,"f : excuse me . can i exchange this dress , please ? i bought it this morning , but i got the wrong size . it is too small for my daughter . m : ok , we have larger sizes , but not in red . what about this blue one ? f : oh , good , she likes that color , too . actually , this blue dress is an even better choice for her birthday present .",train_5872,train,m : well . i 'll help you change your dress to a larger size .,m : all right . i 'll change your dress for another color .,m : well . i 'll help you change your dress to a smaller size .,m : here is the style you like . you just picked the wrong style .,mutual
1,3,"m : did you have a good weekend , anna ? f : yes , it was fantastic . we visited my uncle 's farm , he used to run a small hotel . but he recently decided to raise sheep instead . m : sheep ? i would think that would be hard , with such a low amount of rain .",train_6198,train,"f : yeah , he used to raise sheep , but now he is doing reporting weather for a living .","f : although he used to raise sheep , now he is running a hotel for a living .","f : yes , now he is raising sheep for a living with a lot of rain .","f : yeah , now he is raising sheep for a living .",mutual
2,2,m : john is certainly the funniest person in class . he can always make us laugh . f : really ? i think i still have to get used to his sense of humor .,train_4343,train,m : you used to understand john 's humor . but now you ca n't . what 's wrong with you ?,m : i 'm happy that you really appreciate john 's humor .,m : you ca n't appreciate john 's humor ? how things happened to you ?,m : why you do n't have any sense of humor ?,mutual
3,0,"f : have we done everything on the list ? m : we have n't booked our hotel room yet . but i 'll do that by telephone tonight . f : we were going to look for another chair for the living room , but i 'm too tired to think about that now . let 's have lunch . m : what time is it ? it 's only 12:30. i told howard we 'd meet him a little before 1:00 .",train_3309,train,f : before 1:00. we do n't have much time to prepare . hurry up .,"f : we 've done everything on the list , so let 's meet him before 1pm .","f : the appointment is at 1:00 ? it 's 11:00 , so we still have time .",f : i forgot we 'll meet him after 1:00. i 've just book the hotel room .,mutual
4,0,"m : are you feeling well , linda ? you look very tired . f : no , my sister and i had a birthday party for our brother last night . i did n't get much sleep . m : where did you have the party ? f : at my uncles ' , and the party lasted very long . after the party , i had to type a history paper for the next morning . i was very nervous for my professor hates accepting any late papers .",train_1912,train,"m : it is a wise choice to type your paper after the party , or you will not able to hand it in to your professor on time this morning .","m : it is good that you type your paper after the party , but it is unnecessary because your professor loves to accept late papers .",m : you uncle must be happy because you visited him after your mother 's birthday party . did you bring the birthday cake to him ?,m : no wonder you are still energetic today with sufficient sleep . you studied history after the party and did great in today 's test .,mutual
5,3,"m : which season do you like the most ? f : i like summer the most . but at the same time i hate it . i like summer because it 's warm . i do n't mind the heat and i like the longer days so i can do more activities like swimming , bicycling and many other outdoor activities . i usually take a short vacation during the summer so i always look forward to that . however , the part about summer i hate the most is during the rainy season . when it gets hot along with the rain and the wind , it gets terrible .",train_3928,train,"m : so , you like the heat of summer but you do n't like the longer days ?","m : so , you like the warmth of summer but 

Each example in the dataset has a context composed of a first sentence (in the field `sent1`) and an introduction to the second sentence (in the field `sent2`). Then four possible endings are given (in the fields `ending0`, `ending1`, `ending2` and `ending3`) and the model must pick the right one (indicated in the field `label`). The following function lets us visualize a give example a bit better:

In [9]:
def show_one(example):
    print(f"Context: {example['article']}")
    print(f"  A - {example['option1']}")
    print(f"  B - {example['option2']}")
    print(f"  C - {example['option3']}")
    print(f"  D - {example['option4']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [10]:
show_one(train_dataset[0])

Context: m : do you want to sit under that tree , granny ? it wo n't be so hot there . f : i 'd prefer to sit in the sun . it looks nice there by the river . m : shall i put the chairs there then ? f : yes , there in the car . why do n't you go and get them ?
  A - m : i will , granny . i agree with your idea . let 's sit here by the river .
  B - m : okay . it is cold outside , we just stay in the car .
  C - m : i do n't want to , because it looks bad there by the river .
  D - m : sure . it will be hot under that tree .

Ground truth: option A


In [11]:
# show_one(train_dataset[15])

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [12]:
from transformers import AutoTokenizer, BertTokenizer
tok_checkpoint = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(tok_checkpoint)   
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

c:\users\redux gamer\appdata\local\programs\python\python39\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


We pass along `use_fast=True` to the call above to use one of the fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, but if you got an error with the previous call, remove that argument.

You can directly call this tokenizer on one sentence or a pair of sentences:

In [13]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 8667, 117, 1142, 1141, 5650, 106, 102, 1262, 1142, 5650, 2947, 1114, 1122, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

To preprocess our dataset, we will thus need the names of the columns containing the sentence(s). The following dictionary keeps track of the correspondence task to column names:

We can them write the function that will preprocess our samples. The tricky part is to put all the possible pairs of sentences in two big lists before passing them to the tokenizer, then un-flatten the result so that each example has four input ids, attentions masks, etc.

When calling the `tokenizer`, we use the argument `truncation=True`. This will ensure that an input longer that what the model selected can handle will be truncated to the maximum length accepted by the model.

In [14]:
ending_names = ['option1', 'option2', 'option3', 'option4']

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["article"]]
    # Grab all second sentences possible for each context.
    # question_headers = examples["sent2"]
    second_sentences = [[f"{examples[end][i]}" for end in ending_names] for i, header in enumerate(examples["article"])]
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    del first_sentences, second_sentences
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}



# ending_names = ["ending0", "ending1", "ending2", "ending3"]

# def preprocess_function(examples):
#     # Repeat each first sentence four times to go with the four possibilities of second sentences.
#     first_sentences = [[context] * 4 for context in examples["sent1"]]
#     # Grab all second sentences possible for each context.
#     question_headers = examples["sent2"]
#     second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
#     # Flatten everything
#     first_sentences = sum(first_sentences, [])
#     second_sentences = sum(second_sentences, [])
    
#     # Tokenize
#     tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
#     # Un-flatten
#     return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists of lists for each key: a list of all examples (here 5), then a list of all choices (4) and a list of input IDs (length varying here since we did not apply any padding):

In [15]:
examples = train_dataset[:5]
# print(examples)
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]), [len(x) for x in features["input_ids"][0]])

5 4 [103, 95, 98, 91]


In [16]:
# features.keys()
# datasets['train']

To check we didn't do anything group when grouping all possibilites then unflattening, let's have a look at the decoded inputs for a given example:

In [17]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] m : can i help you, madam? f : yes, i want to report a theft that happened to my house during our absence. m : i see. what is missing? [SEP] f : nothing. i think you should go to the police and report about the theft. [SEP]',
 '[CLS] m : can i help you, madam? f : yes, i want to report a theft that happened to my house during our absence. m : i see. what is missing? [SEP] f : nothing. the hotel that i have been staying had a theft. it was so annoying. [SEP]',
 "[CLS] m : can i help you, madam? f : yes, i want to report a theft that happened to my house during our absence. m : i see. what is missing? [SEP] f : i don't know yet. maybe i'll go back and check later. what else should i do now? [SEP]",
 "[CLS] m : can i help you, madam? f : yes, i want to report a theft that happened to my house during our absence. m : i see. what is missing? [SEP] f : i don't know yet. the hotel that i have been staying had a theft. what should i do now? [SEP]"]

We can compare it to the ground truth:

In [18]:
show_one(train_dataset[3])

Context: m : can i help you , madam ? f : yes , i want to report a theft that happened to my house during our absence . m : i see . what is missing ?
  A - f : nothing . i think you should go to the police and report about the theft .
  B - f : nothing . the hotel that i have been staying had a theft . it was so annoying .
  C - f : i do n't know yet . maybe i 'll go back and check later . what else should i do now ?
  D - f : i do n't know yet . the hotel that i have been staying had a theft . what should i do now ?

Ground truth: option C


This seems alright, so we can apply this function on all the examples in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [19]:
encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
encoded_val_dataset = val_dataset.map(preprocess_function, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our task is about mutliple choice, we use the `AutoModelForMultipleChoice` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [20]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

c:\users\redux gamer\appdata\local\programs\python\python39\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertF

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [21]:
model_name =  model_checkpoint.split("/")[-1]#'dd'
print(os.path.join(os.getcwd(),f"{model_name}-finetuned-mutual-frozen"))
args = TrainingArguments(
    os.path.join(os.getcwd(),f"{model_name}-finetuned-mutual-frozen"),
    evaluation_strategy = "epoch",
    learning_rate=5e-4,
    save_strategy='epoch',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.1,
    push_to_hub=False,
)

D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen


Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-swag"` or `"huggingface/bert-finetuned-swag"`).

Then we need to tell our `Trainer` how to form batches from the pre-processed inputs. We haven't done any padding yet because we will pad each batch to the maximum length inside the batch (instead of doing so with the maximum length of the whole dataset). This will be the job of the *data collator*. A data collator takes a list of examples and converts them to a batch (by, in our case, applying padding). Since there is no data collator in the library that works on our specific problem, we will write one, adapted from the `DataCollatorWithPadding`:

In [22]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

When called on a list of examples, it will flatten all the inputs/attentions masks etc. in big lists that it will pass to the `tokenizer.pad` method. This will return a dictionary with big tensors (of shape `(batch_size * 4) x seq_length`) that we then unflatten.

We can check this data collator works on a list of features, we just have to make sure to remove all features that are not inputs accepted by our model (something the `Trainer` will do automatically for us after):

In [23]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_train_dataset[i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)
# print(batch['attention_mask'])
# model(batch)

Again, all those flatten/un-flatten are sources of potential errors so let's make another sanity check on our inputs:

In [33]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(1)]
print(len([tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(1)][0]
))

1320


In [25]:
show_one(train_dataset[8])

Context: m : hello , do you have the best of mozart ? f : i 'm sorry . we 've just sold out , but we can order one for you . if you give us your number , we 'll call you when the cd arrives .
  A - m : it 's so generous of you to give me the cd for free .
  B - m : thank you , i 'm curious when will the beethoven cd arrives approximately though .
  C - m : thank you , you are really helpful .
  D - m : it will be more helpful if you can tell me when will the beethoven cd arrives approximately .

Ground truth: option C


All good!

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. We need to define a function for this, which will just use the `metric` we loaded earlier, the only preprocessing we have to do is to take the argmax of our predicted logits:

In [26]:
import numpy as np
soft = torch.nn.Softmax(dim=1)
def compute_metrics(eval_predictions):
    
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    all_preds = np.argsort(predictions, axis=1)
#     print('predictions',predictions)
#     blah = []
    predictions   = soft(torch.from_numpy(predictions)) #softmax
#     print('softmasx',predictions.shape)

    print(np.sum(predictions.numpy(),axis=1))
    #     meh = [blah.extend(predictions[all_preds][i]) for i in range(len(predictions[all_preds]))] 
#     meh = [predictions[all_preds][i] for i in range(len(predictions[all_preds]))]
#     blah = np.reshape(predictions,(-1,4))
#     print('meh',meh)
#     exit()

#     print(all_preds,blah)
    r1 = len(preds[(preds == label_ids)]) / len(preds)

    
    copy_predictions = np.copy(predictions)
    x,y = predictions.shape
    index_matrix = np.arange(0, y).repeat(x).reshape(x,y).T == preds.reshape(-1,1)
    copy_predictions[index_matrix] = 0
    
    
    
    
    ###############
#     sorted_by_second = sorted(data, key=lambda tup: tup[1])
    ###############
    r2 = len(copy_predictions[(preds != label_ids)][copy_predictions[(preds != label_ids)].argmax(axis=1) == label_ids[(preds != label_ids)]]) / len(preds)
    # return r2
#     print(copy_predictions.shape)
#     print(preds.shape)
#     print(label_ids.shape)
    
    mrr = (1 / (preds + 1)).sum() / len(preds)
    

    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item(), "@R1":r1, "@R2":r1+r2, "MRR":mrr}
# def compute_metrics(eval_predictions):
#     predictions, label_ids = eval_predictions
#     preds = np.argmax(predictions, axis=1)
#     return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [27]:
# print(encoded_train_dataset[0])
print(train_dataset[0])
# x = np.array([[0.8, 0.2, 0.3, 0.4],
#             [0.1, 0.5, 0.3, 0.4],
#             [0.1, 0.2, 0.7, 0.4],
#             [0.1, 0.2, 0.9, 0.4]])
# labels = np.array([0, 1, 2, 3])
# compute_metrics((x,labels))
# model.classifier = torch.nn.Linear(in_features=768, out_features=1, bias=True)
for params in model.parameters():
      params.requires_grad = False

for index, layer in enumerate(model.bert.encoder.layer):
      if index >=9: #total of 11 bert layers.
            for params in layer.parameters():
                  params.requires_grad = True
    

# for i, param in enumerate(model.bert.parameters()):

#     param.requires_grad = False

# for param in model.classifier.parameters():
#     param.requires_grad = True
    
# print(model.classifier)
# for i, param in enumerate(model.parameters()):
#     print(param)
# print(model)

{'label': 0, 'article': "m : do you want to sit under that tree , granny ? it wo n't be so hot there . f : i 'd prefer to sit in the sun . it looks nice there by the river . m : shall i put the chairs there then ? f : yes , there in the car . why do n't you go and get them ?", 'id': 'train_5792', 'scope': 'train', 'option1': "m : i will , granny . i agree with your idea . let 's sit here by the river .", 'option2': 'm : okay . it is cold outside , we just stay in the car .', 'option3': "m : i do n't want to , because it looks bad there by the river .", 'option4': 'm : sure . it will be hot under that tree .', 'dataset': 'mutual'}


Then we just need to pass all of this along with our datasets to the `Trainer`:

In [28]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [29]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: option2, article, scope, option1, option4, id, option3, dataset.
***** Running training *****
  Num examples = 7088
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4430
c:\users\redux gamer\appdata\local\programs\python\python39\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,@r1,@r2,Mrr
1,1.396800,1.386294,0.234763,0.234763,0.234763,0.527276
2,1.391000,1.386296,0.244921,0.244921,0.244921,0.519564
3,1.391200,1.386296,0.224605,0.224605,0.225734,0.531321
4,1.389700,1.386295,0.237020,0.237020,0.237020,0.517871
5,1.388200,1.386292,0.232506,0.232506,0.232506,0.511569


The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: option2, article, scope, option1, option4, id, option3, dataset.
***** Running Evaluation *****
  Num examples = 886
  Batch size = 8


[1.0000001  1.         0.99999994 1.         1.         1.0000001
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.0000001  1.         1.
 1.         1.         1.         0.99999994 1.         1.
 1.         1.         0.99999994 1.         1.0000001  1.
 1.         1.         1.         0.99999994 1.         1.
 1.         1.         1.         1.         1.0000001  1.
 1.         0.9999999  1.0000001  1.0000001  1.         1.
 1.         0.99999994 1.         1.         0.99999994 1.
 1.         1.         1.         1.         1.         1.
 1.0000001  1.0000001  1.         1.         0.99999994 1.
 1.         1.         1.         1.         0.99999994 0.99999994
 1.         1.0000001  1.         1.         0.99999994 1.
 1.         1.         0.99999994 1.         1.         1.
 1.         1.         1.         1.     

C:\Users\REDUXG~1\AppData\Local\Temp/ipykernel_15348/3671027395.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  index_matrix = np.arange(0, y).repeat(x).reshape(x,y).T == preds.reshape(-1,1)
Saving model checkpoint to D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-886
Configuration saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-886\config.json
Model weights saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-886\pytorch_model.bin
tokenizer config file saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-886\tokenizer_config.json
Special tokens file saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-886\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: option2, article, scope, option1, option4, id, option3, dataset.
***** Run

...

[1.         1.         1.         1.         1.0000001  1.0000001
 1.         1.         1.0000001  1.         1.         0.99999994
 1.         1.         1.0000001  1.         1.0000001  1.0000001
 1.         0.99999994 1.         1.         1.         1.0000001
 0.99999994 1.         1.         1.         1.         1.
 0.99999994 1.         1.         1.         1.         1.
 1.         1.         1.         1.0000001  1.         1.
 0.99999994 1.         1.0000001  1.         1.         1.
 0.9999999  1.         1.         1.         1.         1.
 1.         1.         1.         1.0000001  1.         0.99999994
 1.         1.0000001  1.0000001  1.         1.         0.99999994
 1.         1.         1.         0.9999999  1.         1.
 1.         0.99999994 1.         1.0000001  1.         1.
 0.99999994 1.         1.         1.         0.9999999  1.0000001
 1.         1.         1.         0.99999994 1.         1.0000001
 1.         1.         1.         1.         1.         

C:\Users\REDUXG~1\AppData\Local\Temp/ipykernel_15348/3671027395.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  index_matrix = np.arange(0, y).repeat(x).reshape(x,y).T == preds.reshape(-1,1)
Saving model checkpoint to D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-3544
c:\users\redux gamer\appdata\local\programs\python\python39\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Configuration saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-3544\config.json
Model weights saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-3544\pytorch_model.bin
tokenizer config file saved i

[1.         0.99999994 1.         0.9999999  1.         1.
 1.         1.         1.         0.9999999  1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.99999994 1.         0.9999999  1.
 1.         1.         0.99999994 1.         0.99999994 1.
 1.0000001  1.         0.99999994 1.         1.         1.
 1.         1.         0.9999999  1.         1.         1.
 1.         1.         0.99999994 1.         1.         1.
 1.         1.         1.         1.         0.99999994 0.9999999
 0.99999994 1.         1.         1.         0.99999994 1.
 1.         1.         0.99999994 0.99999994 1.         1.
 1.         0.9999999  0.99999994 1.0000001  1.0000001  0.99999994
 1.         1.         1.         0.99999994 1.         0.99999994
 1.         0.99999994 1.         0.99999994 1.         0.99999994
 1.         1.         1.         0.99999994 0.99999994 1.
 1.         1.         1.         1.         0.99999994 0.9999999
 1.         1.    

C:\Users\REDUXG~1\AppData\Local\Temp/ipykernel_15348/3671027395.py:26: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  index_matrix = np.arange(0, y).repeat(x).reshape(x,y).T == preds.reshape(-1,1)
Saving model checkpoint to D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-4430
c:\users\redux gamer\appdata\local\programs\python\python39\lib\site-packages\transformers\configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Configuration saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-4430\config.json
Model weights saved in D:\dl4nlp\bert-base-uncased-finetuned-mutual-frozen\checkpoint-4430\pytorch_model.bin
tokenizer config file saved i

TrainOutput(global_step=4430, training_loss=1.3911724426407308, metrics={'train_runtime': 616.5331, 'train_samples_per_second': 57.483, 'train_steps_per_second': 7.185, 'total_flos': 1.7774899749633792e+16, 'train_loss': 1.3911724426407308, 'epoch': 5.0})